# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.12.0 opencv-python mediapipe sklearn matplotlib

In [ ]:
pip install tensorflow

In [ ]:
pip install pandas

In [ ]:
pip install pyspark

In [ ]:
pip install Pillow

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([ pose,lh, rh])

In [97]:
import numpy as np

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros((33, 3)).flatten()
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros((468, 3)).flatten()
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros((21, 3)).flatten()
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros((21, 3)).flatten()
    
    # Concatenate the keypoints
    all_keypoints = np.concatenate([pose,lh, rh])
    
    # Format into groups of 3 with curly braces
    formatted_keypoints = ["{" + f"{all_keypoints[i]}, {all_keypoints[i+1]}, {all_keypoints[i+2]}" + "}" for i in range(0, len(all_keypoints), 3)]
    
    return formatted_keypoints


# 5. Collect Keypoint Values for Training and Testing

## ISL_CSLRT_Corpus video Files

# Sample MP processing 

In [91]:
import pandas as pd

# Path to your CSV file (including the file name and extension)
csv_file_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\corpus_csv_files\ISL_CSLRT_Corpus details.csv"

# Read the CSV file into a pandas DataFrame
# Assuming UTF-8 encoding, but adjust if needed
df = pd.read_csv(csv_file_path).head(1)

# Define the root path using raw string literal
rootpath = 'C:/Users/ranjan.patra/OneDrive - Lingaro Sp. z o. o/DATA/IITJ/Course/Projects/ISL/'


In [ ]:
label_prev=""
#sample_num=0
#df_k = pd.DataFrame(columns=['Label', 'sample_type','Sample_Number','frame_shape', 'Frame_Number', 'Keypoints'])
sample_type="Video"
new_width = 1920
new_height = 1080

for index, row in df.iterrows():
    #print("label_prev:",label_prev)
    #print(index)
    label = row["Sentences"]  # Get the "Word" value for the current row
    #print("label:",label)
    if label_prev!=label:
        sample_num=0
    else:
        sample_num=sample_num+1
    print("sample_num:",sample_num)
    label_prev=label
    video_path = rootpath + row["File location"]
    video_path = video_path.replace('/', '\\')
    print("video_path:",row["File location"])

    #for video_file in VIDEO_FILES:
    cap = cv2.VideoCapture(video_path)
    

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        #print("Hi")
        #frame_number = 0
        #with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, image = cap.read()
            if ret is None:
                print("Error reading frame. Exiting loop.")
                break
            elif not ret:
                print("End of video reached. Exiting loop.")
                break

            frame_shape=image.shape
            print("Frame Shape:",frame_shape)
            image=cv2.resize(image, (new_width, new_height))
            print("Frame Shape:",image.shape)
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            image, results = mediapipe_detection(image, holistic)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            keypoints = extract_keypoints(results)
            frame_number=cap.get(cv2.CAP_PROP_POS_FRAMES)
            print("Frame Num:",frame_number)
            print("Frame Num:", frame_number, keypoints)
            

            draw_styled_landmarks(image, results)
            


            cv2.putText(image, 'Collecting frames for "{}"; Sample Number:{}; frame number:{}'.format(label,sample_num,frame_number), (15, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1, cv2.LINE_AA)


            cv2.imshow('OpenCV Feed', image)

            keypoints = extract_keypoints(results)
            #new_data = pd.DataFrame({'Label': [label], 'sample_type': [sample_type],'Sample_Number': [sample_num], 'Frame_Number': [frame_number], 'Keypoints': [keypoints]})

            #new_data = pd.DataFrame({'Label': [label], 'Sample Number': [sample_num],'frame_shape;'[frame_shape], 'Frame Number': [frame_number], 'Keypoints': [keypoints]})
            #df_k = pd.concat([df_k, new_data], ignore_index=True)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

                frame_number += 1

        cap.release()
        print("Cap realease")
        cv2.destroyAllWindows()

#df_k.to_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks")


# Main Processing

Shape of Video changed to 
    new_width = 1920
    new_height = 1080

In [99]:
import pandas as pd

# Path to your CSV file (including the file name and extension)
csv_file_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\corpus_csv_files\ISL_CSLRT_Corpus details.csv"

# Read the CSV file into a pandas DataFrame
# Assuming UTF-8 encoding, but adjust if needed
df = pd.read_csv(csv_file_path)

# Define the root path using raw string literal
rootpath = 'C:/Users/ranjan.patra/OneDrive - Lingaro Sp. z o. o/DATA/IITJ/Course/Projects/ISL/'


In [ ]:
label_prev=""
#sample_num=0
df_k = pd.DataFrame(columns=['Label', 'sample_type','Sample_Number','frame_shape', 'Frame_Number', 'Keypoints'])
sample_type="Video"
new_width = 1920
new_height = 1080


for index, row in df.iterrows():
    #print("label_prev:",label_prev)
    #print(index)
    label = row["Sentences"]  # Get the "Word" value for the current row
    #print("label:",label)
    if label_prev!=label:
        sample_num=0
    else:
        sample_num=sample_num+1
    print("sample_num:",sample_num)
    label_prev=label
    video_path = rootpath + row["File location"]
    video_path = video_path.replace('/', '\\')
    print("video_path:",row["File location"])
    #video_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_DICTIONARY\A\Alien_species.mp4"

    #for video_file in VIDEO_FILES:
    cap = cv2.VideoCapture(video_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        #print("Hi")
        #frame_number = 0
        #with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, image = cap.read()
            if ret is None:
                print("Error reading frame. Exiting loop.")
                break
            elif not ret:
                print("End of video reached. Exiting loop.")
                break

            frame_shape=image.shape
            image=cv2.resize(image, (new_width, new_height))
            #print("Frame Shape:",image.shape)
            #frame_shape=image.shape
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            image, results = mediapipe_detection(image, holistic)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            keypoints = extract_keypoints(results)
            frame_number=cap.get(cv2.CAP_PROP_POS_FRAMES)
            #print("Frame Num:",frame_number)
            #print("Frame Num:", frame_number, keypoints)
            

            #draw_styled_landmarks(image, results)
            


            #cv2.putText(image, 'Collecting frames for {}; Sample Number:{}; frame number:{}'.format(label,sample_num,frame_number), (15, 12),
                            #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1, cv2.LINE_AA)


            #cv2.imshow('OpenCV Feed', image)

            #keypoints = extract_keypoints(results)
            new_data = pd.DataFrame({'Label': [label], 'sample_type': [sample_type],'Sample_Number': [sample_num],'frame_shape':[frame_shape], 'Frame_Number': [frame_number], 'Keypoints': [keypoints]})

            #new_data = pd.DataFrame({'Label': [label], 'Sample Number': [sample_num], 'Frame Number': [frame_number], 'Keypoints': [keypoints]})
            df_k = pd.concat([df_k, new_data], ignore_index=True)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

                frame_number += 1

        cap.release()
        print("Cap realease")
        cv2.destroyAllWindows()

df_k.to_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks_lh_rh")


In [ ]:
cap.release()
print("Cap realease")
cv2.destroyAllWindows()

In [102]:



df_from_parquet = pd.read_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks_lh_rh")

display(df_from_parquet)

,Label,sample_type,Sample_Number,frame_shape,Frame_Number,Keypoints
0,are you free today,Video,0,"[1080, 1920, 3]",1.0,"[{0.47676751017570496, 0.3017379641532898, -0...."
1,are you free today,Video,0,"[1080, 1920, 3]",2.0,"[{0.47681325674057007, 0.30156728625297546, -0..."
2,are you free today,Video,0,"[1080, 1920, 3]",3.0,"[{0.4772616922855377, 0.3015567362308502, -0.6..."
3,are you free today,Video,0,"[1080, 1920, 3]",4.0,"[{0.477945476770401, 0.30148565769195557, -0.6..."
4,are you free today,Video,0,"[1080, 1920, 3]",5.0,"[{0.47859013080596924, 0.30141815543174744, -0..."
...,...,...,...,...,...,...
51558,"you need a medicine, take this one",Video,4,"[1080, 1920, 3]",114.0,"[{0.49276989698410034, 0.35694217681884766, -0..."
51559,"you need a medicine, take this one",Video,4,"[1080, 1920, 3]",115.0,"[{0.4929289221763611, 0.3573005497455597, -0.6..."
51560,"you need a medicine, take this one",Video,4,"[1080, 1920, 3]",116.0,"[{0.49306631088256836, 0.35789814591407776, -0..."
51561,"you need a medicine, take this one",Video,4,"[1080, 1920, 3]",117.0,"[{0.49327489733695984, 0.3583262860774994, -0...."
